### Installation
SQLite comes pre-installed with Python, so you don't need to install it separately. Just import the sqlite3 module:

In [1]:
!pip show tensorflow-metadata

Name: tensorflow-metadata
Version: 1.16.1
Summary: Library and standards for schema and statistics.
Home-page: 
Author: Google Inc.
Author-email: tensorflow-extended-dev@googlegroups.com
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: absl-py, googleapis-common-protos, protobuf
Required-by: tensorflow-datasets


In [2]:
!pip show grpcio-status

Name: grpcio-status
Version: 1.71.0
Summary: Status proto mapping for gRPC
Home-page: https://grpc.io
Author: The gRPC Authors
Author-email: grpc-io@googlegroups.com
License: Apache License 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: googleapis-common-protos, grpcio, protobuf
Required-by: google-cloud-pubsub


In [3]:
!pip install protobuf==<compatible_version>

/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `pip install protobuf==<compatible_version>'


In [4]:
!pip install --force-reinstall mysql-connector-python==8.0.32

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.16.1 requires protobuf<6.0.0dev,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.


In [5]:
import sqlite3

#### Creating a Database Connection
You can create a connection to either an in-memory database or a file-based database:

In [6]:
# In-memory database (data is lost when connection is closed)
conn_memory = sqlite3.connect(':memory:')

# File-based database (data persists after connection is closed)
conn_file = sqlite3.connect('example.db')

# Create a cursor object to execute SQL statements
cursor = conn_memory.cursor()

### Basic Operations
#### Creating Tables
Tables are created using the SQL CREATE TABLE statement:

In [7]:
# Create a table named 'employees'
cursor.execute('''
CREATE TABLE employees (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    department TEXT,
    salary REAL,
    hire_date TEXT
)
''')

# Commit the changes
conn_memory.commit()

#### Inserting Data
You can insert data using the SQL INSERT INTO statement:

In [8]:
# Single row insertion
cursor.execute("INSERT INTO employees (name, department, salary, hire_date) VALUES ('Alice', 'Sales', 60000, '2022-01-15')")

# Multiple row insertion using executemany
employees_data = [
    ('Bob', 'Marketing', 75000, '2022-02-10'),
    ('Charlie', 'Engineering', 90000, '2021-11-01'),
    ('Diana', 'HR', 65000, '2022-03-20'),
    ('Eva', 'Engineering', 95000, '2021-08-15')
]

cursor.executemany("INSERT INTO employees (name, department, salary, hire_date) VALUES (?, ?, ?, ?)", employees_data)

# Commit the changes
conn_memory.commit()

#### Querying Data
Retrieve data using the SQL SELECT statement:

In [9]:
# Fetch all rows
cursor.execute("SELECT * FROM employees")
all_employees = cursor.fetchall()
print("All employees:")
for employee in all_employees:
    print(employee)

# Fetch rows with a condition
cursor.execute("SELECT name, salary FROM employees WHERE department = 'Engineering'")
engineering_employees = cursor.fetchall()
print("\nEngineering employees:")
for employee in engineering_employees:
    print(f"Name: {employee[0]}, Salary: ${employee[1]}")

# Fetch a single row
cursor.execute("SELECT * FROM employees WHERE id = 1")
first_employee = cursor.fetchone()
print("\nFirst employee:", first_employee)

All employees:
(1, 'Alice', 'Sales', 60000.0, '2022-01-15')
(2, 'Bob', 'Marketing', 75000.0, '2022-02-10')
(3, 'Charlie', 'Engineering', 90000.0, '2021-11-01')
(4, 'Diana', 'HR', 65000.0, '2022-03-20')
(5, 'Eva', 'Engineering', 95000.0, '2021-08-15')

Engineering employees:
Name: Charlie, Salary: $90000.0
Name: Eva, Salary: $95000.0

First employee: (1, 'Alice', 'Sales', 60000.0, '2022-01-15')


#### Updating and Deleting Data
Modify existing data with UPDATE and DELETE statements:

In [10]:
# Update a record
cursor.execute("UPDATE employees SET salary = 65000 WHERE name = 'Alice'")

# Delete a record
cursor.execute("DELETE FROM employees WHERE name = 'Diana'")

# Commit the changes
conn_memory.commit()

# Verify the changes
cursor.execute("SELECT * FROM employees")
updated_employees = cursor.fetchall()
print("\nAfter updates:")
for employee in updated_employees:
    print(employee)


After updates:
(1, 'Alice', 'Sales', 65000.0, '2022-01-15')
(2, 'Bob', 'Marketing', 75000.0, '2022-02-10')
(3, 'Charlie', 'Engineering', 90000.0, '2021-11-01')
(5, 'Eva', 'Engineering', 95000.0, '2021-08-15')


### Intermediate Operations
#### Using SQLite with Pandas
SQLite works seamlessly with pandas, allowing you to read SQL queries directly into DataFrames:

In [11]:
import pandas as pd

# Read query results into a DataFrame
df = pd.read_sql_query("SELECT * FROM employees", conn_memory)
print("\nEmployees DataFrame:")
print(df)

# Calculate statistics
print("\nSalary statistics:")
print(df['salary'].describe())

# Group by department
dept_stats = df.groupby('department').agg({
    'salary': ['mean', 'min', 'max', 'count']
})
print("\nDepartment statistics:")
print(dept_stats)

# Write DataFrame back to SQLite
df_new = pd.DataFrame({
    'name': ['Frank', 'Grace'],
    'department': ['Sales', 'Marketing'],
    'salary': [70000, 72000],
    'hire_date': ['2022-05-01', '2022-04-15']
})

df_new.to_sql('employees', conn_memory, if_exists='append', index=False)


Employees DataFrame:
   id     name   department   salary   hire_date
0   1    Alice        Sales  65000.0  2022-01-15
1   2      Bob    Marketing  75000.0  2022-02-10
2   3  Charlie  Engineering  90000.0  2021-11-01
3   5      Eva  Engineering  95000.0  2021-08-15

Salary statistics:
count        4.000000
mean     81250.000000
std      13768.926368
min      65000.000000
25%      72500.000000
50%      82500.000000
75%      91250.000000
max      95000.000000
Name: salary, dtype: float64

Department statistics:
              salary                        
                mean      min      max count
department                                  
Engineering  92500.0  90000.0  95000.0     2
Marketing    75000.0  75000.0  75000.0     1
Sales        65000.0  65000.0  65000.0     1


2

### Transactions
Transactions ensure that a series of operations are atomic (all succeed or all fail):

In [12]:
try:
    # Start a transaction
    conn_memory.execute("BEGIN TRANSACTION")

    # Perform multiple operations
    conn_memory.execute("UPDATE employees SET salary = salary * 1.1 WHERE department = 'Sales'")
    conn_memory.execute("INSERT INTO employees (name, department, salary, hire_date) VALUES ('Helen', 'Finance', 85000, '2022-06-01')")

    # Simulate an error (uncomment to test rollback)
    # raise Exception("Simulated error")

    # Commit the transaction
    conn_memory.execute("COMMIT")
    print("\nTransaction committed successfully")

except Exception as e:
    # Roll back on error
    conn_memory.execute("ROLLBACK")
    print(f"\nTransaction rolled back due to error: {e}")


Transaction committed successfully


### Using SQLite with Python Functions
You can create custom SQLite functions using Python:

In [13]:
# Register a custom function for calculating years of service
def years_of_service(hire_date):
    from datetime import datetime
    if hire_date:
        hire_date = datetime.strptime(hire_date, '%Y-%m-%d')
        today = datetime.now()
        return (today - hire_date).days / 365.25
    return None

# Register the function with SQLite
conn_memory.create_function("YEARS_OF_SERVICE", 1, years_of_service)

# Use the custom function in a query
cursor.execute("""
SELECT name, department, hire_date, ROUND(YEARS_OF_SERVICE(hire_date), 1) as years
FROM employees
ORDER BY years DESC
""")

print("\nYears of service:")
for row in cursor.fetchall():
    print(f"{row[0]} ({row[1]}): {row[2]} - {row[3]} years")


Years of service:
Eva (Engineering): 2021-08-15 - 3.6 years
Charlie (Engineering): 2021-11-01 - 3.4 years
Alice (Sales): 2022-01-15 - 3.2 years
Bob (Marketing): 2022-02-10 - 3.1 years
Grace (Marketing): 2022-04-15 - 3.0 years
Frank (Sales): 2022-05-01 - 2.9 years
Helen (Finance): 2022-06-01 - 2.8 years


### Foreign Keys and Relationships
Create related tables with foreign key constraints:

In [14]:
# Enable foreign key support
conn_memory.execute("PRAGMA foreign_keys = ON")

# Create departments table
cursor.execute('''
CREATE TABLE departments (
    id INTEGER PRIMARY KEY,
    name TEXT UNIQUE NOT NULL,
    location TEXT
)
''')

# Create a new employees table with a foreign key
cursor.execute('''
CREATE TABLE employees_new (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    department_id INTEGER,
    salary REAL,
    hire_date TEXT,
    FOREIGN KEY (department_id) REFERENCES departments (id)
)
''')

# Insert departments
departments_data = [
    (1, 'Sales', 'New York'),
    (2, 'Marketing', 'Los Angeles'),
    (3, 'Engineering', 'San Francisco'),
    (4, 'HR', 'Chicago'),
    (5, 'Finance', 'Boston')
]

cursor.executemany("INSERT INTO departments (id, name, location) VALUES (?, ?, ?)", departments_data)

# Insert employees with department_id
employees_with_dept = [
    ('Alice', 1, 65000, '2022-01-15'),
    ('Bob', 2, 75000, '2022-02-10'),
    ('Charlie', 3, 90000, '2021-11-01'),
    ('Eva', 3, 95000, '2021-08-15'),
    ('Frank', 1, 70000, '2022-05-01'),
    ('Grace', 2, 72000, '2022-04-15'),
    ('Helen', 5, 85000, '2022-06-01')
]

cursor.executemany("INSERT INTO employees_new (name, department_id, salary, hire_date) VALUES (?, ?, ?, ?)", employees_with_dept)

# Join tables to get employee and department information
cursor.execute('''
SELECT e.name, d.name as department, d.location, e.salary
FROM employees_new e
JOIN departments d ON e.department_id = d.id
ORDER BY e.salary DESC
''')

print("\nEmployees with department details:")
for row in cursor.fetchall():
    print(f"{row[0]} works in {row[1]} ({row[2]}) with salary ${row[3]}")


Employees with department details:
Eva works in Engineering (San Francisco) with salary $95000.0
Charlie works in Engineering (San Francisco) with salary $90000.0
Helen works in Finance (Boston) with salary $85000.0
Bob works in Marketing (Los Angeles) with salary $75000.0
Grace works in Marketing (Los Angeles) with salary $72000.0
Frank works in Sales (New York) with salary $70000.0
Alice works in Sales (New York) with salary $65000.0


### Advanced Operations
#### Indexes for Performance
Create indexes to improve query performance:

In [15]:
# Create an index on the department_id column
cursor.execute("CREATE INDEX idx_employees_department ON employees_new(department_id)")

# Create a composite index on department_id and salary
cursor.execute("CREATE INDEX idx_dept_salary ON employees_new(department_id, salary)")

# Measure query performance before and after indexing
import time

# Without using index
start_time = time.time()
cursor.execute("SELECT * FROM employees_new WHERE salary > 80000")
cursor.fetchall()
print(f"\nQuery without optimized index: {time.time() - start_time:.6f} seconds")

# Using index
start_time = time.time()
cursor.execute("SELECT * FROM employees_new WHERE department_id = 3 AND salary > 80000")
cursor.fetchall()
print(f"Query with optimized index: {time.time() - start_time:.6f} seconds")


Query without optimized index: 0.000168 seconds
Query with optimized index: 0.000210 seconds


### Full-Text Search
Set up full-text search for efficient text searching:

In [16]:
# Check if the table exists before creating it
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='employee_fts'")
table_exists = cursor.fetchone()

# Create the table only if it doesn't exist
if not table_exists:
    cursor.execute('''
    CREATE VIRTUAL TABLE employee_fts USING fts5(
        name,
        department,
        bio,
        content='employees_new',
        content_rowid='id'
    )
    ''')
    print("Table 'employee_fts' created successfully.")
else:
    print("Table 'employee_fts' already exists.")

# ... (rest of your code) ...

Table 'employee_fts' created successfully.


### BLOB Storage
Store binary data like images in SQLite:

In [17]:
# Create a table for storing files
cursor.execute('''
CREATE TABLE files (
    id INTEGER PRIMARY KEY,
    filename TEXT NOT NULL,
    file_type TEXT NOT NULL,
    data BLOB NOT NULL,
    upload_date TEXT NOT NULL
)
''')

# Function to insert a file into the database
def insert_file(filename, file_type, file_data):
    from datetime import datetime
    upload_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    cursor.execute('INSERT INTO files (filename, file_type, data, upload_date) VALUES (?, ?, ?, ?)',
                  (filename, file_type, file_data, upload_date))
    conn_memory.commit()
    return cursor.lastrowid

# Function to retrieve a file from the database
def get_file(file_id):
    cursor.execute('SELECT filename, file_type, data, upload_date FROM files WHERE id = ?', (file_id,))
    return cursor.fetchone()

# Example: Generate some binary data (simulating an image)
sample_data = b'\x89PNG\r\n\x1a\n' + bytes([0] * 1000)  # Fake PNG header + data

# Store the file
file_id = insert_file('sample.png', 'image/png', sample_data)
print(f"\nFile stored with ID: {file_id}")

# Retrieve the file
file_info = get_file(file_id)
print(f"Retrieved file: {file_info[0]}, Type: {file_info[1]}, Size: {len(file_info[2])} bytes, Uploaded: {file_info[3]}")


File stored with ID: 1
Retrieved file: sample.png, Type: image/png, Size: 1008 bytes, Uploaded: 2025-03-29 12:27:47


### Use Cases
#### Simple Application Database
Example of using SQLite for a simple task manager application:

In [35]:
# Create a task manager database
tasks_conn = sqlite3.connect('tasks.db')
tasks_cursor = tasks_conn.cursor()

# Create tables
tasks_cursor.execute('''
CREATE TABLE IF NOT EXISTS categories (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL UNIQUE
)
''')

tasks_cursor.execute('''
CREATE TABLE IF NOT EXISTS tasks (
    id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    description TEXT,
    due_date TEXT,
    priority INTEGER DEFAULT 0,
    completed INTEGER DEFAULT 0,
    category_id INTEGER,
    created_at TEXT DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (category_id) REFERENCES categories (id)
)
''')

# Add sample data
categories = [('Work',), ('Personal',), ('Study',), ('Health',)]
tasks_cursor.executemany("INSERT OR IGNORE INTO categories (name) VALUES (?)", categories)

tasks = [
    ('Complete SQLite tutorial', 'Finish the comprehensive guide', '2023-03-30', 2, 0, 3),
    ('Prepare presentation', 'Create slides for team meeting', '2023-03-25', 3, 0, 1),
    ('Gym workout', '30 minutes cardio + strength training', '2023-03-24', 1, 0, 4),
    ('Buy groceries', 'Milk, eggs, bread, vegetables', '2023-03-23', 2, 1, 2)
]

tasks_cursor.executemany('''
INSERT INTO tasks (title, description, due_date, priority, completed, category_id)
VALUES (?, ?, ?, ?, ?, ?)
''', tasks)

tasks_conn.commit()

# Task manager functions
def add_task(title, description, due_date, priority, category_id):
    tasks_cursor.execute('''
    INSERT INTO tasks (title, description, due_date, priority, category_id)
    VALUES (?, ?, ?, ?, ?)
    ''', (title, description, due_date, priority, category_id))
    tasks_conn.commit()
    return tasks_cursor.lastrowid

def get_tasks(completed=None, category_id=None):
    query = "SELECT t.id, t.title, t.description, t.due_date, t.priority, t.completed, c.name as category "
    query += "FROM tasks t LEFT JOIN categories c ON t.category_id = c.id WHERE 1=1 "
    params = []

    if completed is not None:
        query += "AND t.completed = ? "
        params.append(completed)

    if category_id is not None:
        query += "AND t.category_id = ? "
        params.append(category_id)

    query += "ORDER BY t.due_date, t.priority DESC"

    tasks_cursor.execute(query, params)
    return tasks_cursor.fetchall()

def mark_completed(task_id, completed=1):
    tasks_cursor.execute("UPDATE tasks SET completed = ? WHERE id = ?", (completed, task_id))
    tasks_conn.commit()
    return tasks_cursor.rowcount

# Demo the task manager
print("\nTask Manager Demo:")
print("\nAll tasks:")
for task in get_tasks():
    status = "✓" if task[5] else "□"
    print(f"{status} {task[1]} (Due: {task[3]}, Priority: {task[4]}, Category: {task[6]})")

print("\nAdding a new task...")
new_task_id = add_task("Buy birthday gift", "Gift for mom's birthday", "2023-03-27", 2, 2)
print(f"New task added with ID: {new_task_id}")

print("\nMarking a task as completed...")
mark_completed(1)
print("Task 1 marked as completed")

print("\nPending tasks:")
for task in get_tasks(completed=0):
    print(f"□ {task[1]} (Due: {task[3]}, Priority: {task[4]}, Category: {task[6]})")

# Close the connection when done
tasks_conn.close()


Task Manager Demo:

All tasks:
✓ Buy groceries (Due: 2023-03-23, Priority: 2, Category: Personal)
□ Gym workout (Due: 2023-03-24, Priority: 1, Category: Health)
□ Prepare presentation (Due: 2023-03-25, Priority: 3, Category: Work)
□ Complete SQLite tutorial (Due: 2023-03-30, Priority: 2, Category: Study)

Adding a new task...
New task added with ID: 5

Marking a task as completed...
Task 1 marked as completed

Pending tasks:
□ Gym workout (Due: 2023-03-24, Priority: 1, Category: Health)
□ Prepare presentation (Due: 2023-03-25, Priority: 3, Category: Work)
□ Buy birthday gift (Due: 2023-03-27, Priority: 2, Category: Personal)


### Data Analysis Pipeline
Using SQLite as part of a data analysis pipeline:

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

# Create a database for sales data
sales_conn = sqlite3.connect('sales_analysis.db')

# Create tables for sales data
sales_conn.execute('''
CREATE TABLE IF NOT EXISTS regions (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    country TEXT NOT NULL
)
''')

sales_conn.execute('''
CREATE TABLE IF NOT EXISTS products (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    category TEXT NOT NULL,
    price REAL NOT NULL
)
''')

sales_conn.execute('''
CREATE TABLE IF NOT EXISTS sales (
    id INTEGER PRIMARY KEY,
    date TEXT NOT NULL,
    product_id INTEGER NOT NULL,
    region_id INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    FOREIGN KEY (product_id) REFERENCES products (id),
    FOREIGN KEY (region_id) REFERENCES regions (id)
)
''')

# Insert sample data
regions = [
    (1, 'Northeast', 'USA'),
    (2, 'Southeast', 'USA'),
    (3, 'Midwest', 'USA'),
    (4, 'West', 'USA'),
    (5, 'Central', 'Canada'),
    (6, 'Eastern', 'Canada')
]

products = [
    (1, 'Laptop', 'Electronics', 1200),
    (2, 'Smartphone', 'Electronics', 800),
    (3, 'Tablet', 'Electronics', 500),
    (4, 'Desk Chair', 'Furniture', 250),
    (5, 'Desk', 'Furniture', 350),
    (6, 'Bookshelf', 'Furniture', 150)
]

# Generate random sales data
np.random.seed(42)
sales_data = []
sale_id = 1

for month in range(1, 13):
    for day in range(1, 29, 7):  # Weekly data
        for product_id in range(1, 7):
            for region_id in range(1, 7):
                # Random quantity between 1 and 20
                quantity = np.random.randint(1, 21)
                date = f"2022-{month:02d}-{day:02d}"
                sales_data.append((sale_id, date, product_id, region_id, quantity))
                sale_id += 1

# Insert data
sales_conn.executemany("INSERT INTO regions VALUES (?, ?, ?)", regions)
sales_conn.executemany("INSERT INTO products VALUES (?, ?, ?, ?)", products)
sales_conn.executemany("INSERT INTO sales VALUES (?, ?, ?, ?, ?)", sales_data)
sales_conn.commit()

# Query for analysis
sales_query = '''
SELECT
    s.date,
    r.name as region,
    r.country,
    p.name as product,
    p.category,
    p.price,
    s.quantity,
    p.price * s.quantity as revenue
FROM sales s
JOIN products p ON s.product_id = p.id
JOIN regions r ON s.region_id = r.id
ORDER BY s.date
'''

# Load into pandas
sales_df = pd.read_sql_query(sales_query, sales_conn)

# Basic analysis
print("\nSales Data Analysis:")
print(f"Total records: {len(sales_df)}")
print(f"Date range: {sales_df['date'].min()} to {sales_df['date'].max()}")
print(f"Total revenue: ${sales_df['revenue'].sum():,.2f}")

# Monthly revenue
sales_df['date'] = pd.to_datetime(sales_df['date'])
sales_df['month'] = sales_df['date'].dt.strftime('%Y-%m')
monthly_revenue = sales_df.groupby('month')['revenue'].sum().reset_index()

print("\nMonthly Revenue:")
for _, row in monthly_revenue.iterrows():
    print(f"{row['month']}: ${row['revenue']:,.2f}")

# Revenue by category
category_revenue = sales_df.groupby('category')['revenue'].sum().reset_index()
print("\nRevenue by Category:")
for _, row in category_revenue.iterrows():
    print(f"{row['category']}: ${row['revenue']:,.2f}")

# Revenue by region
region_revenue = sales_df.groupby(['region', 'country'])['revenue'].sum().reset_index()
print("\nRevenue by Region:")
for _, row in region_revenue.iterrows():
    print(f"{row['region']} ({row['country']}): ${row['revenue']:,.2f}")

# Store aggregate data back in SQLite
monthly_revenue.to_sql('monthly_revenue', sales_conn, if_exists='replace', index=False)
category_revenue.to_sql('category_revenue', sales_conn, if_exists='replace', index=False)
region_revenue.to_sql('region_revenue', sales_conn, if_exists='replace', index=False)

# Close the connection
sales_conn.close()


Sales Data Analysis:
Total records: 1728
Date range: 2022-01-01 to 2022-12-22
Total revenue: $9,568,500.00

Monthly Revenue:
2022-01: $772,950.00
2022-02: $781,550.00
2022-03: $782,700.00
2022-04: $845,200.00
2022-05: $730,200.00
2022-06: $797,200.00
2022-07: $748,300.00
2022-08: $781,550.00
2022-09: $815,500.00
2022-10: $816,850.00
2022-11: $878,550.00
2022-12: $817,950.00

Revenue by Category:
Electronics: $7,325,800.00
Furniture: $2,242,700.00

Revenue by Region:
Central (Canada): $1,550,900.00
Eastern (Canada): $1,653,100.00
Midwest (USA): $1,596,950.00
Northeast (USA): $1,607,850.00
Southeast (USA): $1,567,400.00
West (USA): $1,592,300.00


### Embedded Database
Example of using SQLite as an embedded database for a simple IoT application:

In [40]:
import sqlite3
import datetime
import random
import time

# Create a database for sensor data
iot_db = sqlite3.connect('iot_sensors.db')

# Create tables
iot_db.execute('''
CREATE TABLE IF NOT EXISTS devices (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    location TEXT,
    type TEXT NOT NULL,
    active INTEGER DEFAULT 1
)
''')

iot_db.execute('''
CREATE TABLE IF NOT EXISTS sensor_data (
    id INTEGER PRIMARY KEY,
    device_id INTEGER NOT NULL,
    timestamp TEXT NOT NULL,
    temperature REAL,
    humidity REAL,
    pressure REAL,
    light_level REAL,
    FOREIGN KEY (device_id) REFERENCES devices (id)
)
''')

# Insert sample devices
devices = [
    (1, 'Living Room Sensor', 'Living Room', 'Environmental', 1),
    (2, 'Kitchen Sensor', 'Kitchen', 'Environmental', 1),
    (3, 'Outdoor Sensor', 'Backyard', 'Weather', 1),
    (4, 'Bedroom Sensor', 'Bedroom', 'Environmental', 1)
]

iot_db.executemany("INSERT OR IGNORE INTO devices VALUES (?, ?, ?, ?, ?)", devices)
iot_db.commit()

# Function to simulate sensor readings
def generate_sensor_reading(device_id):
    now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # Base values depending on device type and location
    if device_id == 1:  # Living Room
        temp_base, humid_base, press_base, light_base = 22.0, 45.0, 1013.0, 60.0
    elif device_id == 2:  # Kitchen
        temp_base, humid_base, press_base, light_base = 23.5, 50.0, 1013.0, 70.0
    elif device_id == 3:  # Outdoor
        temp_base, humid_base, press_base, light_base = 18.0, 65.0, 1012.0, 90.0
    elif device_id == 4:  # Bedroom
        temp_base, humid_base, press_base, light_base = 21.0, 40.0, 1013.0, 30.0
    else:
        temp_base, humid_base, press_base, light_base = 20.0, 50.0, 1013.0, 50.0

    # Add random variations
    temperature = temp_base + random.uniform(-2.0, 2.0)
    humidity = humid_base + random.uniform(-5.0, 5.0)
    pressure = press_base + random.uniform(-1.0, 1.0)
    light_level = light_base + random.uniform(-10.0, 10.0)

    return (None, device_id, now, temperature, humidity, pressure, light_level)

# Simulate data collection over a period
print("\nSimulating IoT data collection...")

# Insert some historical data
historical_data = []
now = datetime.datetime.now()

for day in range(7):  # Past week
    for hour in range(0, 24, 3):  # Every 3 hours
        timestamp = (now - datetime.timedelta(days=day, hours=hour)).strftime('%Y-%m-%d %H:%M:%S')
        for device_id in range(1, 5):
            reading = generate_sensor_reading(device_id)
            # Replace timestamp with historical one
            historical_data.append((None, device_id, timestamp) + reading[3:])

# Add the data to the database
iot_db.executemany('''
INSERT INTO sensor_data VALUES (?, ?, ?, ?, ?, ?, ?)
''', historical_data)
iot_db.commit()

# Simulate real-time data collection
print("Starting real-time sensor data collection...")
for i in range(5):  # Collect 5 readings in real-time
    for device_id in range(1, 5):
        reading = generate_sensor_reading(device_id)
        iot_db.execute('''
        INSERT INTO sensor_data VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', reading)
    iot_db.commit()
    print(f"Collected readings from all sensors (iteration {i+1}/5)")
    time.sleep(1)  # Simulate 1-second delay between readings

# Query the data for analysis
cursor = iot_db.cursor()

# Get the latest reading for each device
print("\nLatest sensor readings:")
cursor.execute('''
SELECT d.name, s.timestamp, s.temperature, s.humidity, s.pressure, s.light_level
FROM sensor_data s
JOIN devices d ON s.device_id = d.id
WHERE s.timestamp = (
    SELECT MAX(timestamp) FROM sensor_data WHERE device_id = s.device_id
)
''')

latest_readings = cursor.fetchall()
for reading in latest_readings:
    print(f"{reading[0]} ({reading[1]}): Temp: {reading[2]:.1f}°C, Humidity: {reading[3]:.1f}%, Pressure: {reading[4]:.1f} hPa, Light: {reading[5]:.1f}%")

# Calculate average values by device
print("\nAverage values by device:")
cursor.execute('''
SELECT d.name,
       AVG(s.temperature) as avg_temp,
       AVG(s.humidity) as avg_humid,
       AVG(s.pressure) as avg_press,
       AVG(s.light_level) as avg_light
FROM sensor_data s
JOIN devices d ON s.device_id = d.id
GROUP BY s.device_id
''')

avg_readings = cursor.fetchall()
for reading in avg_readings:
    print(f"{reading[0]}: Avg Temp: {reading[1]:.1f}°C, Avg Humidity: {reading[2]:.1f}%, Avg Pressure: {reading[3]:.1f} hPa, Avg Light: {reading[4]:.1f}%")

# Find temperature anomalies (more than 2 degrees from the average)
print("\nTemperature anomalies:")
for device_id in range(1, 5):
    cursor.execute('''
    SELECT d.name, s.timestamp, s.temperature,
           (SELECT AVG(temperature) FROM sensor_data WHERE device_id = ?) as avg_temp
    FROM sensor_data s
    JOIN devices d ON s.device_id = d.id
    WHERE s.device_id = ? AND ABS(s.temperature - (SELECT AVG(temperature) FROM sensor_data WHERE device_id = ?)) > 2
    ORDER BY s.timestamp DESC
    LIMIT 5
    ''', (device_id, device_id, device_id))

    anomalies = cursor.fetchall()
    if anomalies:
        for anomaly in anomalies:
            print(f"{anomaly[0]} at {anomaly[1]}: Temp {anomaly[2]:.1f}°C (Average: {anomaly[3]:.1f}°C)")

# Close the database connection
iot_db.close()


Simulating IoT data collection...
Starting real-time sensor data collection...
Collected readings from all sensors (iteration 1/5)
Collected readings from all sensors (iteration 2/5)
Collected readings from all sensors (iteration 3/5)
Collected readings from all sensors (iteration 4/5)
Collected readings from all sensors (iteration 5/5)

Latest sensor readings:
Living Room Sensor (2025-03-29 12:35:18): Temp: 22.7°C, Humidity: 41.5%, Pressure: 1013.5 hPa, Light: 64.8%
Kitchen Sensor (2025-03-29 12:35:18): Temp: 23.1°C, Humidity: 51.8%, Pressure: 1012.2 hPa, Light: 69.9%
Outdoor Sensor (2025-03-29 12:35:18): Temp: 16.1°C, Humidity: 67.9%, Pressure: 1012.0 hPa, Light: 85.1%
Bedroom Sensor (2025-03-29 12:35:18): Temp: 20.4°C, Humidity: 38.0%, Pressure: 1012.0 hPa, Light: 35.5%

Average values by device:
Living Room Sensor: Avg Temp: 21.9°C, Avg Humidity: 44.8%, Avg Pressure: 1013.1 hPa, Avg Light: 59.3%
Kitchen Sensor: Avg Temp: 23.4°C, Avg Humidity: 50.1%, Avg Pressure: 1013.0 hPa, Avg L

### Performance Optimization
Modern SQLite installations are highly optimized, but there are additional steps you can take to improve performance:

#### PRAGMA Settings

In [41]:
import sqlite3

# Create a new connection
conn = sqlite3.connect('optimized.db')

# Speed up transactions with WAL mode
conn.execute('PRAGMA journal_mode = WAL')

# Synchronous setting (0 = OFF, 1 = NORMAL, 2 = FULL, 3 = EXTRA)
# Lower values increase speed but with higher risk of corruption in case of system failure
conn.execute('PRAGMA synchronous = NORMAL')

# Increase cache size (in pages)
conn.execute('PRAGMA cache_size = 10000')

# Enable memory-mapped I/O (can improve performance for read-heavy applications)
conn.execute('PRAGMA mmap_size = 30000000000')

# Temporarily disable foreign keys during bulk inserts
conn.execute('PRAGMA foreign_keys = OFF')

# Batch inserts in a transaction
conn.execute('BEGIN TRANSACTION')
# ... many insert operations ...
conn.execute('COMMIT')

# Re-enable foreign keys
conn.execute('PRAGMA foreign_keys = ON')

# Close the connection
conn.close()